# Router Continued

## High-Level Overview

The code captures network packets, processes them at the TCP/IP level, stores payloads, and then, once a certain threshold (window size) is reached, splits and reorders the payload data into multiple packets before sending them out. The main steps and logic are:

1. **Packet Capture and Queueing:**  
   Packets are captured using `libpcap` and passed to `packet_handler_from`. Instead of immediately sending them, the code queues and processes them, extracting payload data and storing it into a shared payload buffer.

2. **TCP Processing and Payload Accumulation:**  
   For TCP packets:
   - Parse Ethernet, IP, and TCP headers.
   - Log TCP flags and options (and remove SACK if present).
   - Extract the payload from the packet and append it to an `internal->payload_buffer`.
   - Once the `payload_buffer` reaches a certain "window size," the code triggers `window()` to handle the entire collected payload.

3. **Window Logic (Payload Splitting and Reordering):**  
   When enough payload data accumulates (exceeding `internal->window_size`):
   - The `window()` function is called.
   - It splits the payload data into multiple packets of random sizes (above a minimum size).
   - Reorders these packets (e.g., reverses their order) to simulate out-of-order delivery.
   - Reassembles the headers and updates sequence numbers for each packet.
   - Sends each packet out through the raw socket, adding delays to simulate network conditions.
   - Logs throughput and timing statistics.

4. **Serialization, Logging, and Checksums:**  
   As before, it logs packet info, increments counters for TCP flags/options, and ensures all checksums (IP and TCP) are correctly computed after modifications.

---

## Conceptual Diagram (Text-Based)

```
            +------------------------+
            |    Packet Capture     |
            |  (packet_handler_from)|
            +-----------+------------+
                        |
                        v
                  +-----+-----+   add payload to 
                  | queue_packet|--> payload_buffer  
                  +-----------+-+   
                              |
                           window size  
                           exceeded?
                              | yes
                              v
                       +-------------+
                       |   window()  |
                       |   Process,  |
                       |   Split,    |
                       |   Reorder   |
                       +------+------+
                              |
                       send_packet() for each chunk
                              |
                              v
                         +---------+
                         | raw socket
                         +---------+
```

**Explanation:**  
- Incoming packets are queued and processed. Once enough payload accumulates, `window()` splits, reorders, and sends the newly formed packets.

---

## Key Complex Snippets and Their Explanations

### Snippet 1: Accumulating Payload and Triggering Window

```c++
// Inside queue_packet after TCP processing:

if (packet_data.length > total_header_length) {
    size_t payload_length = packet_data.length - total_header_length;
    unsigned char *payload = packet_data.data.data() + total_header_length;

    {
        std::lock_guard<std::mutex> lock(internal->payload_buffer_mutex);
        internal->payload_buffer.insert(internal->payload_buffer.end(), payload, payload + payload_length);

        if (internal->payload_buffer.size() >= internal->window_size) {
            // Once we have enough payload, process the window
            window(internal, packet_data);
        }
    }
}
```

**What’s Happening:**  
After extracting headers, if there's payload data, the code accumulates it in `internal->payload_buffer`. When the buffer reaches a predefined `window_size`, it calls `window()` to handle the entire buffered payload at once. This simulates scenarios like batching data before sending, or enforcing a certain window-based sending strategy.

**Why it’s Complex:**  
It requires careful synchronization and ensuring that once the buffer reaches a certain threshold, the correct function (`window()`) is triggered. Also, payloads must be extracted from already processed packets, respecting IP and TCP headers.

---

### Snippet 2: Splitting Payload and Reordering Packets in `window()`

```c++
// Splitting payload into random-sized chunks:
std::vector<size_t> chunk_sizes = split_payload_randomly(total_payload_size, MIN_PAYLOAD_SIZE);

// Creating packet info and reordering:
auto packets_info = create_packet_info(chunk_sizes);
reorder_packets(packets_info);
```

**What’s Happening:**  
- `split_payload_randomly` breaks the large payload into several smaller chunks, each at least `MIN_PAYLOAD_SIZE` in length, but otherwise randomly sized.
- `create_packet_info` records offset and size for each chunk.
- `reorder_packets` then reverses or rearranges the sequence of these packets, creating out-of-order delivery conditions.

**Why it’s Complex:**  
This simulates network conditions where packets may not arrive in the order they were sent. It's complexity lies in ensuring each chunk is correctly referenced by offset and size, and that the resulting reordered packets are valid.

---

### Snippet 3: Rebuilding Packets with New Sequence Numbers

```c++
auto new_packet = build_packet(original_packet, internal->payload_buffer, pkt.offset, pkt.size, seq_number);
```

**What’s Happening in `build_packet()`:**  
- Constructs a new packet from the original headers plus a subset of the payload data.
- Adjusts IP total length and sets a fresh TCP sequence number (`seq_number`).
- Clears TCP options and recomputes checksums to ensure the new packet is valid.

**Why it’s Complex:**  
Raw packet manipulation requires exact offsets, correct header fields, proper endianness, and accurate checksum computations. The code must handle all these details correctly to produce a valid IP/TCP packet.

---

### Snippet 4: Timing, Throughput, and Delays

```c++
std::this_thread::sleep_for(std::chrono::milliseconds(10)); // Delay between sends
```

**What’s Happening:**  
The code introduces delays between sending packets to simulate network conditions. It also measures total time and computes throughput at the end.

**Why it’s Complex:**  
Simulating network conditions isn't just about sending packets. Adding realistic timing and logging throughput requires careful recording of start/end times and computing rates.

---

## Conclusion

This enhanced code goes beyond mere packet capture and forwarding. It introduces:

- **Payload accumulation:** Gathering payloads until they reach a configurable window size.
- **Payload manipulation:** Splitting the accumulated payload into multiple random-sized chunks.
- **Packet reordering and rewriting:** Creating new packets from the split payload, altering sequence numbers, removing TCP options, and ensuring all is properly checksummed.
- **Throughput and timing measurement:** Logging how fast data is processed and sent.

The key complexity lies in meticulous low-level packet manipulation, combined with concurrency control, timing, and simulation of complex network behaviors (out-of-order delivery, delayed sending, etc.).